In [ ]:
import json
import os
import pandas as pd
import re
import requests

In [ ]:
parl_api_url = 'https://lop.parl.ca/ParlinfoWebAPI'
accept_json = {'Accept': 'application/json'}

In [ ]:
def drop_time(df):
    date_cols = [col for col in df.columns if col.endswith('Date')]
    for col in date_cols:
        df[col] = df[col].str[:10]

def drop_french(df):
    to_drop = [col for col in df.columns if col.endswith('Fr')]
    df.drop(to_drop, axis=1, inplace=True)
    
def drop_empty_cols(df):
    to_drop = [col for col in df.columns if df[col].count() == 0]
    df.drop(to_drop, axis=1, inplace=True)
    
def cleanup(df):
    drop_time(df)
    drop_french(df)
    drop_empty_cols(df)

In [ ]:
r = requests.get(parl_api_url + '/Person/SearchAndRefine?refiners=4-1,', headers=accept_json)
d = r.json()
len(d)

In [ ]:
r = requests.get(parl_api_url + '/Person/SearchAndRefine?refiners=28-1,28-2,28-3,', headers=accept_json)
d = r.json()
len(d)

In [ ]:
len(d[0]['Roles'])

In [ ]:
rows = [{k: (p[k] if k in p else None) for k in ['PersonId', 'LastName', 'UsedFirstName', 'ProvincialExperienceEN', 'MunicipalExperienceEn']} for p in d]
len(rows)

In [ ]:
df = pd.DataFrame(rows)
df

In [ ]:
df['ProvincialExperienceEN'].value_counts()

In [ ]:
df[df['LastName'] == 'Aglukkaq']

In [ ]:
r = requests.get(parl_api_url + '/Person/GetPersonWebProfile/4487', headers=accept_json)
d = r.json()
len(d)

In [ ]:
sorted(d.keys())

In [ ]:
for k in list(d.keys()):
    if k.endswith('Fr'):
        del d[k]

In [ ]:
person = d['Person']
name = f"{person['UsedFirstName']} {person['LastName']}"
name = person['DisplayName']
name

In [ ]:
filename = name.replace(' ', '_') + '.json'
filename = name + '.json'
filename

In [ ]:
with open(filename, 'w') as f:
    json.dump(d, f, indent=2)

In [ ]:
!code $filename

In [ ]:
sorted(d.keys())

In [ ]:
sorted(d['Person'].keys())

In [ ]:
[key for key in sorted(d['Person']['Roles'][0].keys()) if not key.endswith('Fr')]

In [ ]:
current_parliament = 43
dfs = []
for parliament in range(1, current_parliament + 1):
    df = pd.read_csv('../data/parliaments/parliament-%d-people.csv' % parliament)
    df.insert(0, 'parliament', parliament)
    dfs.append(df)
df = pd.concat(dfs)
df.count()

In [ ]:
df.to_csv('../data/parliaments/all_parliaments.csv', index=False, encoding='utf8')

In [ ]:
df.nunique().sort_values(ascending=False)

In [ ]:
df['PersonId'].nunique()

In [ ]:
df['StraightDisplayName'].nunique()

In [ ]:
def num_prefix(str):
    return int(str.split('-')[0])

def name_suffix(str):
    return str.split('-')[-1]

In [ ]:
# Match people .json files
pattern = re.compile('[0-9]+-.+\.json$')
people_dir = '../data/people'
files = sorted(filter(lambda d: pattern.match(d), os.listdir(people_dir)), key=name_suffix)
len(files)

In [ ]:
def person_recs(n = None):
    for file in files[:n] if n else files:
        with open(os.path.join(people_dir, file)) as f:
            yield json.load(f)

In [ ]:
person_cols = ['PersonId', 'LastName', 'UsedFirstName']
role_cols = ['PersonRoleId', 'ParliamentNumber', 'PartyEn', 'ToBeStyledAsEn',
             'OrganizationTypeEn', 'OrganizationLongEn', 'GroupingTitleEn', 'PortFolioEn', 'NameEn', 
             'IsMP', 'IsSenator', 'IsActing',
             'StartDate', 'StartDateIsApproximate', 
             'EndDate', 'EndDateIsApproximate', 'NotesEn']

rows = []
for rec in person_recs():
    person = rec['Person']
    person_tuple = tuple(person[col] for col in person_cols)
    for role in rec['FederalExperience']:
        role_tuple = tuple(role[col] for col in role_cols)
        row = person_tuple + role_tuple
        rows.append(row)
len(rows)


In [ ]:
person_cols = ['PersonId', 'LastName', 'UsedFirstName']
role_cols = ['PersonRoleId', 'ParliamentNumber', 'PartyEn', 'ToBeStyledAsEn',
             'OrganizationTypeEn', 'OrganizationLongEn', 'GroupingTitleEn', 'PortFolioEn', 'NameEn', 
             'IsMP', 'IsSenator', 'IsActing',
             'StartDate', 'StartDateIsApproximate', 
             'EndDate', 'EndDateIsApproximate', 'EndReasonEn', 'EndReasonTypeEn', 
             'NotesEn']

rows = []
for rec in person_recs():
    person = rec['Person']
    person_tuple = tuple(person[col] for col in person_cols)
    for role in rec['FederalExperience']:
        role_tuple = tuple(role[col] for col in role_cols)
        row = person_tuple + role_tuple
        rows.append(row)
len(rows)

In [ ]:
df = pd.DataFrame(rows, columns = person_cols + role_cols) \
  .sort_values(['LastName', 'UsedFirstName', 'PersonId', 'StartDate', 'PersonRoleId'])
df = df.drop_duplicates()
len(df)

In [ ]:
cleanup(df)
df.sample(5)

In [ ]:
df.loc[39471]

In [ ]:
df['ToBeStyledAsEn'].value_counts()[:30]

In [ ]:
filename = os.path.join(people_dir, 'fed_roles.csv')
df.to_csv(filename, index=False, encoding='utf8')

In [ ]:
!open $filename

In [ ]:
person_cols = ['PersonId', 'LastName', 'UsedFirstName']
education_cols = ['SchoolNameLongEn', 'FieldOfStudyEn', 'DiplomaLongEn', 'GraduationYear']

rows = []
for rec in person_recs():
    person = rec['Person']
    person_tuple = tuple(person[col] for col in person_cols)
    if person['Education']:
        for edu in person['Education']:
            edu_tuple = tuple(edu[col] for col in education_cols)
            row = person_tuple + edu_tuple
            rows.append(row)
len(rows)


In [ ]:
df = pd.DataFrame(rows, columns = person_cols + education_cols) \
  .sort_values(['LastName', 'UsedFirstName', 'PersonId', 'GraduationYear'])
df = df.drop_duplicates()
len(df)

In [ ]:
df.sample(25)

In [ ]:
filename = os.path.join(people_dir, 'education.csv')
df.to_csv(filename, index=False, encoding='utf8')

In [ ]:
education_cols

In [ ]:
parl_num = 42
parl = df[df['ParliamentNumber'] == parl_num]
parl

In [ ]:
filename = os.path.join(people_dir, f'parl_{parl_num}_roles.csv')
parl42.to_csv(filename, index=False, encoding='utf8')

In [ ]:
!open $filename

In [ ]:
mp_roles = df[df['IsMP']].drop_duplicates()
len(mp_roles)

In [ ]:
mp_roles.nunique().sort_values(ascending=False)

In [ ]:
mp_roles.fillna('').groupby(['GroupingTitleEn', 'OrganizationTypeEn', 'PortFolioEn', 'NameEn'])[['PersonRoleId']].count()

In [ ]:
mp_roles.fillna('').groupby(['GroupingTitleEn', 'OrganizationTypeEn']).nunique()

In [ ]:
mp_roles.nunique()

In [ ]:
mp_roles = mp_roles.drop(['GroupingTitleEn', 'PortFolioEn', 'NameEn', 'IsMP', 'IsSenator', 'IsActing'], axis=1)
mp_roles

In [ ]:
mp_roles['PersonRoleId'].value_counts()[:25]

In [ ]:
cols = ['PersonId', 'LastName', 'UsedFirstName', 'PersonRoleId', 'StartDate', 'EndDate',
        'PartyEn', 'OrganizationTypeEn', 'OrganizationLongEn']
mp_roles = mp_roles[cols]
mp_roles

In [ ]:
mp_roles.to_csv(os.path.join(people_dir, 'mp_roles.csv'), index=False, encoding='utf8')

In [ ]:
df.columns

In [ ]:
df['GroupingTitleEn'].fillna('').value_counts()

In [ ]:
df['OrganizationTypeEn'].fillna('').value_counts()

In [ ]:
df['IsMP'].fillna('').value_counts()

In [ ]:
parl_roles = df[df['GroupingTitleEn'] == 'Parliamentarian']
parl_roles.count()

In [ ]:
mps_df = df[df['IsMP']]
mps_df[['GroupingTitleEn', 'OrganizationTypeEn', 'OrganizationLongEn', 'GroupingTitleEn', 'PortFolioEn', 'NameEn', 'IsMP']].nunique()

In [ ]:
mps_df['GroupingTitleEn'].value_counts()

In [ ]:
mps_df['OrganizationTypeEn'].value_counts()

In [ ]:
mps_df['OrganizationLongEn'].value_counts()

In [ ]:
mps_df['PortFolioEn'].value_counts()

In [ ]:
mps_df['NameEn'].value_counts()

In [ ]:
parl_roles.fillna('') \
  .groupby(['OrganizationTypeEn', 'OrganizationLongEn', 'GroupingTitleEn', 'PortFolioEn', 'NameEn', 'IsMP'])[['PersonRoleId']].count() \
  [:25]

In [ ]:
df['PartyEn'].value_counts()

In [ ]:
df['IsActing'].value_counts()

In [ ]:
df.groupby(['EndReasonTypeEn', 'EndReasonEn'])[['PersonRoleId']].count()

In [ ]:
df[df['EndReasonTypeEn'] == 'Resignation']

In [ ]:
df2 = df[~df['EndReasonTypeEn'].isna() | ~df['EndReasonEn'].isna()]
df2

In [ ]:
df[['EndReasonEn', 'EndReasonTypeEn']].drop_duplicates()

In [ ]:
df.fillna('na').pivot_table(
    index=['EndReasonTypeEn', 'EndReasonEn'],
    values=['PartyEn', 'PersonId', 'PersonRoleId'],
    aggfunc='nunique'
)[['PartyEn', 'PersonId', 'PersonRoleId']]

In [ ]:
df.fillna('na').groupby(['EndReasonTypeEn', 'EndReasonEn'])[['PartyEn', 'PersonId', 'PersonRoleId']].nunique() \
  .sort_values('PersonRoleId', ascending=False)

In [ ]:
df[df['EndReasonEn'] == 'Resigned between May 30, 1930 and September 9,1930.']

In [ ]:
row = df.loc[107319]
row

In [ ]:
row = 107319
df.loc[row, 'NotesEn'] = df.loc[row, 'EndReasonEn']
df.loc[row, 'EndReasonEn'] = None

In [ ]:
df['PersonRoleId'].count(), df['PersonRoleId'].nunique()

In [ ]:
df[:50]

In [ ]:
df[:50].sort_values(['LastName', 'UsedFirstName', 'StartDate', 'PersonRoleId'])

In [ ]:
len(df)

In [ ]:
path = os.path.join(people_dir, 'person_roles.csv')
df.to_csv(path, index=False, encoding='utf8')

# Candidates
From https://lop.parl.ca/sites/ParlInfo/default/en_CA/ElectionsRidings/Elections

In [ ]:
r = requests.get(parl_api_url + '/Parliament/GetCandidates', headers=accept_json)
d = r.json()
len(d)

In [ ]:
candidates = pd.DataFrame(d)
candidates.count()

In [ ]:
cleanup(candidates)
candidates.count()

In [ ]:
candidates = candidates.sort_values(['ParliamentNumber', 'ProvinceEn', 'ConstituencyEn', 'Votes', 'DisplayName'])
candidates[:10]

In [ ]:
# Constituencies with ties
wins = candidates[candidates['ResultLongEn'] == 'Elected']
cols = ['ParliamentNumber', 'ElectionId', 'IsGeneral', 'ElectionDate', 'ProvinceEn', 'ConstituencyId', 'ConstituencyEn', 'Votes', 'ResultLongEn']
grp = wins.groupby(cols, as_index=False)[['DisplayName']].count()
ties = grp[grp['DisplayName'] > 1]
ties.sort_values('Votes', ascending=False)

In [ ]:
wins[(wins['ElectionId'] == 527) & (wins['ConstituencyId'] == 6971)]

In [ ]:
candidates.to_csv('../data/parliaments/candidates.csv', index=False, encoding='utf8')

# Roles v2

In [ ]:
with open(os.path.join(people_dir, 'parlinfo_28-1_28-2_28-3.json')) as f:
    people = json.load(f)

In [100]:
person = people[0]
sorted(person.keys())

['Age',
 'AssistantCriticOfEn',
 'AssistantCriticOfFr',
 'AssistantDeputySpeakerChairEn',
 'AssistantDeputySpeakerChairFr',
 'AssociateMinisterOfEn',
 'AssociateMinisterOfFr',
 'Bibliography',
 'CabinetChairEn',
 'CabinetChairFr',
 'CabinetCommitteeMemberEn',
 'CabinetCommitteeMemberFr',
 'CityOfBirthEn',
 'CityOfBirthFr',
 'ConstituencyAddresses',
 'ConstituencyEn',
 'ConstituencyFr',
 'CountryOfBirthEn',
 'CountryOfBirthFr',
 'CriticOfEn',
 'CriticOfFr',
 'CurrentConstituencyEn',
 'CurrentConstituencyFr',
 'CurrentPartyEn',
 'CurrentPartyFr',
 'DateOfBirth',
 'DateOfBirthIsApproximate',
 'Death',
 'DeputyHouseLeaderEn',
 'DeputyHouseLeaderFr',
 'DeputyPrimeMinister',
 'DeputySpeaker',
 'DeputyWhipEn',
 'DeputyWhipFr',
 'DiedInOffice',
 'DisplayName',
 'DoNotDiscloseBirthLocation',
 'DoNotDiscloseDateOfBirth',
 'Education',
 'ElectionCandidates',
 'EthnicityLongEn',
 'EthnicityLongFr',
 'ExternalNotesEn',
 'ExternalNotesFr',
 'FamilyRelations',
 'FormalFirstName',
 'Gender',
 'HOCPers

In [101]:
[k for k, v in person.items() if type(v) in [list, dict]]

['Professions',
 'FamilyRelations',
 'Pictures',
 'Roles',
 'Death',
 'YearsOfServiceSegments']

In [ ]:
person_cols = ['PersonId', 'LastName', 'UsedFirstName']
rows = []
for rec in person_recs():
    person = rec['Person']
    person_props = {col: person[col] for col in person_cols}
    for role in person['Roles']:
        role_props = {k: v for k, v in role.items() if not (k.endswith('Fr') or type(v) in [list, dict])}
        row = {**person_props, **role_props}
        rows.append(row)
len(rows)

In [ ]:
sorted(rec.keys())

In [ ]:
sorted(rec['Person'].keys())

In [ ]:
rows[0]

In [ ]:
sorted(rows[0].keys())

In [103]:
df = pd.DataFrame(rows)
cleanup(df)
df.count()

PersonId                  202575
LastName                  202575
UsedFirstName             202575
PersonRoleId              202575
StartDate                 200744
StartDateIsApproximate    202575
EndDate                   198138
EndDateIsApproximate      202575
EndReasonEn                  230
EndReasonTypeEn             1685
SourceOfInformationEn       1139
IsActing                  202575
ActingTextEn                  11
HasCrossedTheFloor        202575
OrganizationId            202575
OrganizationLongEn        202575
OrganizationShortEn       196184
OrganizationAcronymEn     190713
OrganizationTypeId        202575
OrganizationTypeEn        202547
OrganizationProvinceEn      6958
OrganizationHasProfile    202575
IsSenatorialDivision      202575
ToBeStyledAsEn            180918
GroupId                   202575
Source                      8535
PortFolioEn                 7426
GroupingTitleEn            31193
GroupingOrder             202575
ParliamentStart            25827
Parliament

In [128]:
df.sample(25).sort_values(['LastName', 'UsedFirstName', 'StartDate'])

,PersonId,LastName,UsedFirstName,PersonRoleId,StartDate,StartDateIsApproximate,EndDate,EndDateIsApproximate,EndReasonEn,EndReasonTypeEn,...,PartyEn,PartyStartDate,PartyEndDate,IsMP,IsSenator,NotesEn,IsCurrent,RoleId,NameEn,Ordinal
1219,958,Aikins,James Albert Manning,30400,1911-11-15,False,1915-04-15,False,None,None,...,None,None,None,False,False,None,False,2998,Caucus Member,NaN
9011,15678,Baker,George,267854,1989-04-03,False,1991-05-12,False,None,None,...,Liberal Party of Canada,1972-10-30,2017-09-03,False,False,None,False,226,Member,300.0
17458,12709,Blain,Richard,226686,1903-03-12,False,1903-10-24,False,None,None,...,Conservative (1867-1942),1900-11-07,1926-11-27,False,False,None,False,226,Member,300.0
19360,14748,Borden,Frederick William,231209,1896-08-19,False,1896-10-05,False,None,None,...,Liberal Party of Canada,1896-07-30,1911-09-20,False,False,None,False,226,Member,300.0
26307,13311,Buchanan,William Ashbury,186153,1932-10-06,False,1933-05-27,False,None,None,...,Liberal Party of Canada,1925-09-05,1954-07-11,False,False,None,False,226,Member,300.0
33078,6895,Carter,Chesley William,140835,1966-01-18,False,1967-05-08,False,None,None,...,Liberal Party of Canada,1949-06-27,1977-07-28,False,False,None,False,226,Member,300.0
42605,4154,Copp,Arthur Bliss,211731,1915-02-04,False,1915-04-15,False,None,None,...,Liberal Party of Canada,1915-02-01,1917-12-16,False,False,None,False,226,Member,300.0
50679,6281,Denis,Azellus,281462,1976-10-12,False,1977-10-17,False,None,None,...,Liberal Party of Canada,1935-10-14,1991-09-04,False,False,None,False,226,Member,300.0
61317,12772,Fairweather,Robert Gordon Lee,19061,1974-02-27,False,1974-05-09,False,None,None,...,Progressive Conservative Party,1962-06-18,1977-08-31,False,False,None,False,226,Member,300.0
78364,11908,Griesbach,William Antrobus,183562,1943-01-28,False,1944-01-26,False,None,None,...,Conservative (1867-1942),1921-09-15,1945-01-21,False,False,None,False,226,Member,300.0


In [120]:
filename = os.path.join(people_dir, 'roles_with_provincial.csv')
df.to_csv(filename, index=False, encoding='utf8')

In [121]:
!wc $filename

  202594 2361918 58958014 ../data/people/roles_with_provincial.csv


In [123]:
zip_filename = filename + '.zip'
!rm $zip_filename
!zip $zip_filename $filename

rm: ../data/people/roles_with_provincial.csv.zip: No such file or directory
  adding: ../data/people/roles_with_provincial.csv (deflated 92%)


In [124]:
!wc $zipfilename

^C


In [126]:
!ls -al $zip_filename

-rw-r--r--  1 nedgar  staff  4930803  3 May 14:17 ../data/people/roles_with_provincial.csv.zip
